In [8]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_8.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    return max_corr if max_corr >= threshold else 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename):
    submatrix = pd.DataFrame(index=columns, columns=columns)
    for col1, col2 in product(columns, repeat=2):
        submatrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    submatrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix{filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


#filename_sub = "8_sub_mat_3.csv"
#compute_submatrix(data, group_3, filename_sub)

filename_cross = "8_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross)


submatrix8_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,0.336284,0.252467,0.10114,0.270415,0.407715,0.440338,0.370517,0.435784,0.494606,0.311412,...,0.224732,0.202191,0.291736,0.261483,0.328434,0.368466,0.420536,0.398382,0.385623,0.2887
21085.0,0.234894,0.183366,0.203427,0.249667,0.409883,0.394762,0.309464,0.428858,0.435393,0.285556,...,0.180251,0.185205,0.198901,0.197137,0.247432,0.291039,0.288759,0.301076,0.235986,0.18034
21087.0,0.334489,0.297254,0.275392,0.276249,0.345939,0.339469,0.325247,0.368157,0.350769,0.328107,...,0.260201,0.231076,0.230193,0.270025,0.330706,0.288352,0.279211,0.392247,0.320357,0.22424
21089.0,0.285164,0.21745,0.079272,0.145269,0.321338,0.435604,0.411521,0.37988,0.469137,0.279004,...,0.34218,0.2404,0.146054,0.292783,0.308292,0.357869,0.33646,0.349975,0.373039,0.346986
21091.0,0,0,0,0,0.32815,0.195344,0.183029,0.165853,0.273071,0.138653,...,0.065084,0.229347,0.143702,0,0,0.108035,0.480223,0.252007,0.063738,0.366469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.673631,0.748051,0.367921,0.35193,0.19041,0.553245,0.341238,0.421707,0.416232,0.566173,...,0.560674,0.339961,0.343093,0.559677,0.676411,0.660707,0.161672,0.4491,0.612394,0.444629
38091.0,0.534473,0.557286,0.394244,0.308856,0.208529,0.469831,0.317397,0.331406,0.281537,0.449833,...,0.43206,0.285927,0.26982,0.493874,0.509299,0.448921,0.157381,0.399046,0.402047,0.264044
38093.0,0.745938,0.757115,0.497137,0.426087,0.285907,0.539424,0.258378,0.473201,0.261316,0.55443,...,0.638632,0.296085,0.438552,0.631918,0.668983,0.598264,0.203631,0.478283,0.596432,0.300886
38095.0,0.479842,0.507615,0.341523,0.259538,0.188485,0.250225,0.147664,0.19993,0,0.390124,...,0.364598,0.318222,0.378208,0.413111,0.442077,0.311426,0.10916,0.38055,0.285595,0.277954
